## 사용자의 답변과 감정 분석 결과를 
## Gemini 모델에 전달하여 상황에 맞는 추가 질문을 생성

In [1]:
import sys
import os

sys.path.append(os.path.abspath(".."))  # 현재 디렉터리의 상위 경로 추가
#print(sys.path)

In [2]:
from config.api_keys import gemini_key
import google.generativeai as genai

c:\Users\user\miniconda3\envs\DL311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Gemini API 설정
genai.configure(api_key=gemini_key)

# Gemini Pro 모델 선택
model = genai.GenerativeModel('gemini-2.0-flash')

def generate_question(user_response, emotion, conversation):
    prompt = f"""
    사용자의 답변: "{user_response}"
    사용자의 감정: "{emotion}"
    이전 대화 내용: "{conversation}"

    위 정보를 바탕으로 사용자의 상황과 감정을 고려하여 다음 질문을 생성하세요.
    질문은 구체적이고 답변하기 쉬워야 합니다.
    """
    response = model.generate_content(prompt)
    return response.text

In [4]:
user_response = "오늘 너무 힘들었어요."
emotion = "슬픔"
conversation = "사용자: 오늘 기분이 어떠세요? \nLLM: 오늘 너무 힘들었어요."

follow_up_question = generate_question(user_response, emotion, conversation)
print(follow_up_question)

오늘 어떤 일 때문에 그렇게 힘드셨나요? 혹시 구체적으로 말씀해주실 수 있을까요? (예: 업무, 인간관계, 건강 등)



## 전체 대화 내용을 Gemini 모델에 전달하여 간결하게 요약

In [ ]:
# Gemini API 설정
genai.configure(api_key=gemini_key)

# Gemini Pro 모델 선택
model = genai.GenerativeModel('gemini-2.0-flash')

def summarize_conversation(conversation):
    prompt = f"""
    다음 대화 내용을 요약하세요:
    "{conversation}"
    요약은 간결하고 핵심 내용을 포함해야 합니다.
    """
    response = model.generate_content(prompt)
    return response.text

In [6]:
conversation = """
사용자: 오늘 기분이 어떠세요?
LLM: 오늘 너무 힘들었어요.
사용자: 왜 힘드셨어요?
LLM: 회사에서 일이 너무 많았어요.
"""

summary = summarize_conversation(conversation)
print(summary)

LLM은 회사 일 때문에 힘든 하루를 보냈다.

